<a href="https://colab.research.google.com/github/Iconic-Aman/upScaling_Image/blob/master/UpscaleImage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import sys
sys.path.append("/content/repository/ESRGAN")


In [ ]:
!git clone https://github.com/xinntao/ESRGAN

Cloning into 'ESRGAN'...
remote: Enumerating objects: 225, done.
remote: Total 225 (delta 0), reused 0 (delta 0), pack-reused 225
Receiving objects: 100% (225/225), 24.86 MiB | 15.64 MiB/s, done.
Resolving deltas: 100% (86/86), done.


In [ ]:
!ls

ESRGAN	sample_data


In [ ]:
pip install numpy opencv-python

In [ ]:
import os.path as osp
import glob
import cv2
import numpy as np
import torch
import sys
sys.path.append("/content/ESRGAN")

import RRDBNet_arch as arch



model_path = '/content/ESRGAN/models/RRDB_ESRGAN_x4.pth'  # models/RRDB_ESRGAN_x4.pth OR models/RRDB_PSNR_x4.pth
device = torch.device('cuda')  # if you want to run on CPU, change 'cuda' -> cpu
# device = torch.device('cpu')

test_img_folder = '/content/ESRGAN/LR/*'

model = arch.RRDBNet(3, 3, 64, 23, gc=32)
model.load_state_dict(torch.load(model_path), strict=True)
model.eval()
model = model.to(device)

print('Model path {:s}. \nTesting...'.format(model_path))

idx = 0
for path in glob.glob(test_img_folder):
    idx += 1
    base = osp.splitext(osp.basename(path))[0]
    print(idx, base)
    # read images
    img = cv2.imread(path, cv2.IMREAD_COLOR)
    img = img * 1.0 / 255
    img = torch.from_numpy(np.transpose(img[:, :, [2, 1, 0]], (2, 0, 1))).float()
    img_LR = img.unsqueeze(0)
    img_LR = img_LR.to(device)

    with torch.no_grad():
        output = model(img_LR).data.squeeze().float().cpu().clamp_(0, 1).numpy()
    output = np.transpose(output[[2, 1, 0], :, :], (1, 2, 0))
    output = (output * 255.0).round()
    cv2.imwrite('/content/ESRGAN/results/{:s}_rlt.png'.format(base), output)


Model path /content/ESRGAN/models/RRDB_ESRGAN_x4.pth. 
Testing...
1 istockphoto-1061123944-170667a
2 unnamed
3 photo-1518599904199-0ca897819ddb
4 pexels-photo-210186
5 images34
6 95e6a4d45259a8d14aec55abeded310f_screen
7 ferrari-488-front-uhd-8k-wallpaper-548x308
8 images
9 peakpx


OutOfMemoryError: ignored

In [ ]:
!